In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn(input_shape, num_classes, num_conv_layers, num_filters, filter_size, num_dense_layers, num_dense_units, 
               optimizer='adam', learning_rate=0.001, dropout_rate_conv=0.0, dropout_rate_dense=0.0):
    """
    Create a CNN with variable parameters.

    Parameters:
    - input_shape: Tuple, the shape of the input data (e.g., (height, width, channels)).
    - num_classes: int, the number of output classes.
    - num_conv_layers: int, the number of convolutional layers.
    - num_filters: int or List[int], the number of filters for each convolutional layer.
    - filter_size: int or List[int], the size of filters for each convolutional layer.
    - num_dense_layers: int, the number of dense (fully connected) layers.
    - num_dense_units: int or List[int], the number of units for each dense layer.
    - optimizer: str or tf.keras.optimizers.Optimizer, the optimizer to use (default is 'adam').
    - learning_rate: float, the learning rate for the optimizer (default is 0.001).
    - dropout_rate_conv: float, the dropout rate for convolutional layers (default is 0.0).
    - dropout_rate_dense: float, the dropout rate for dense layers (default is 0.0).

    Returns:
    - model: a Keras model.
    """
    model = models.Sequential()

    # Add convolutional layers
    if isinstance(num_filters, list):
        assert len(num_filters) == num_conv_layers, "Number of filters should match the number of convolutional layers"
    else:
        num_filters = [num_filters] * num_conv_layers

    if isinstance(filter_size, list):
        assert len(filter_size) == num_conv_layers, "Filter sizes should match the number of convolutional layers"
    else:
        filter_size = [filter_size] * num_conv_layers

    for i in range(num_conv_layers):
        model.add(layers.Conv2D(num_filters[i], (filter_size[i], filter_size[i]), activation='relu', input_shape=input_shape))
        model.add(layers.MaxPooling2D((2, 2)))
        if dropout_rate_conv > 0.0:
            model.add(layers.Dropout(dropout_rate_conv))

    # Flatten the output before dense layers
    model.add(layers.Flatten())

    # Add dense layers
    if isinstance(num_dense_units, list):
        assert len(num_dense_units) == num_dense_layers, "Number of units should match the number of dense layers"
    else:
        num_dense_units = [num_dense_units] * num_dense_layers

    for i in range(num_dense_layers):
        model.add(layers.Dense(num_dense_units[i], activation='relu'))
        if dropout_rate_dense > 0.0:
            model.add(layers.Dropout(dropout_rate_dense))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model with optimizer and learning rate
    if isinstance(optimizer, str):
        optimizer = tf.keras.optimizers.get(optimizer)
    
    optimizer.learning_rate = learning_rate

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Example usage:
from config import input_shape,num_classes,num_conv_layers, num_filters, filter_size, num_dense_layers, num_dense_units, optimizer, learning_rate, dropout_rate_conv, dropout_rate_dense

cnn_model = create_cnn(input_shape, num_classes, num_conv_layers, num_filters, filter_size, num_dense_layers, num_dense_units, optimizer, learning_rate, dropout_rate_conv, dropout_rate_dense)

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 512)               6